In [53]:
import pandas as pd
import numpy as np
import time

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor    


In [49]:
data = pd.read_pickle('ALLDATA.pkl')

In [50]:
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]
data_y = pd.DataFrame(index=range( data.shape[0]),columns=['Status','Survival'])
data_y['Survival']=data['Wait Time (s)'].values
data_y["Status"] = pd.DataFrame([True] * data.shape[0]).values   #all pedestrians cross, so no right censored data
data_x=data.loc[:,['Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Clear', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Day', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_40 - 49', 'Age_50 - 59', 'Age_60+', 'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'Education_kid', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'shopwalk_kid', 'Vrexp_Yes', 'Heart_Currently',
                   'Heart_Over the years', 'vision_Currently', 'vision_Currently;Over the years', 
                   'vision_Over the years', 'anxiety_Currently', 'anxiety_Over the years', 'Headaches_Currently',
                   'Headaches_Over the years', 'dizziness_Over the years']]       #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later

In [51]:
status = data_y["Status"]
survival = data_y['Survival']
data_y = np.zeros(data.shape[0], dtype={'names':('Status', 'Survival'),
                          'formats':('bool', 'f8')})

data_y['Status'] = status
data_y['Survival'] = survival

In [56]:
def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [58]:
data_x=data_x.apply(pd.to_numeric, errors='coerce')
data_x=calculate_vif_(data_x, thresh=5.0)

/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'Full Braking Before Impact_-1.0' at index: 5
dropping 'Clear' at index: 8
dropping 'One way' at index: 9
dropping 'Day' at index: 11
dropping 'Age_9-12' at index: 13
dropping 'Age_18 - 24' at index: 15
dropping 'Occupation_Employed' at index: 21
dropping 'Occupation_kid' at index: 23
dropping 'Education_Bachelors degree' at index: 23
dropping 'Education_kid' at index: 28
dropping 'mode_Bike' at index: 29
dropping 'workwalk_No' at index: 32
dropping 'shopwalk_Yes' at index: 36
dropping 'AV' at index: 4
dropping 'Lane Width' at index: 1
dropping 'Speed Limit' at index: 0
dropping 'driving license_Yes' at index: 25
dropping 'Minimum Gap' at index: 0
dropping 'Mean Arrival Rate' at index: 0
dropping 'mode_Car' at index: 23
dropping 'numcars' at index: 7
dropping 'Heart_Over the years' at index: 31
Remaining variables:
Index(['Full Braking Before Impact_1', 'Full Braking Before Impact_2',
       'Full Braking Before Impact_3', 'Snowy', 'two way',
       'Two way with median', 'Nig

In [59]:
estimator = CoxPHSurvivalAnalysis()
estimator.fit(data_x,data_y)

CoxPHSurvivalAnalysis(alpha=0, n_iter=100, tol=1e-09, verbose=0)

In [60]:
estimator.score(data_x, data_y)

0.5940847838760723

In [61]:
def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = CoxPHSurvivalAnalysis(alpha=0.000001,n_iter=100, tol=1e-09, verbose=0)
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

scores = fit_and_score_features(data_x.values, data_y)
pd.Series(scores, index=data_x.columns).sort_values(ascending=False)

Age_30 - 39                             0.536548
Occupation_Student                      0.522318
Education_High school diploma           0.520144
Two way with median                     0.518122
shopwalk_No                             0.516514
workwalk_Yes                            0.514533
Age_50 - 59                             0.513162
mode_Public Transit                     0.512536
Education_Masters degree                0.511592
Gender_Female                           0.510568
Full Braking Before Impact_1            0.510389
Headaches_Over the years                0.507856
vision_Over the years                   0.507032
Age_40 - 49                             0.506626
Education_Doctorate degree              0.506580
Education_College/University student    0.506384
Vrexp_Yes                               0.505919
Snowy                                   0.505850
Full Braking Before Impact_2            0.505284
Headaches_Currently                     0.505247
anxiety_Currently   

In [22]:

#data_x=data_x.loc[:,pd.Series(scores, index=data_x.columns)[scores>=0.5].index]   #covariates that have no effect are removed



ValueError: Length of passed values is 61, index implies 53

In [62]:
pipe = Pipeline([('select', SelectKBest(fit_and_score_features, k=3)),
                 ('model', CoxPHSurvivalAnalysis(alpha=0.000001,n_iter=100, tol=1e-09, verbose=10))])

In [63]:
starttime=time.time()

param_grid = {'select__k': np.arange(1,data_x.shape[1]+1)}
gcv = GridSearchCV(pipe, param_grid, return_train_score=True)

gcv.fit(data_x, data_y)

endtime=time.time()


timeelapsed=endtime-starttime

/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


iter      1: update = [0.65296434]
iter      1: loss = 6.3860825337
iter      2: update = [0.21807322]
iter      2: loss = 6.3845721139
iter      3: update = [0.0217759]
iter      3: loss = 6.3845601122
iter      4: update = [0.00019979]
iter      4: loss = 6.3845601112
iter      4: optimization converged
iter      1: update = [-0.32088462]
iter      1: loss = 6.4023695079
iter      2: update = [0.01304118]
iter      2: loss = 6.4023493211
iter      3: update = [1.46143536e-05]
iter      3: loss = 6.4023493211
iter      3: optimization converged
iter      1: update = [-0.39857686]
iter      1: loss = 6.3976940897
iter      2: update = [0.02781648]
iter      2: loss = 6.3976076007
iter      3: update = [0.0001075]
iter      3: loss = 6.3976075995
iter      3: optimization converged
iter      1: update = [ 0.62220575 -0.069986  ]
iter      1: loss = 6.3855819481
iter      2: update = [0.21983592 0.00494314]
iter      2: loss = 6.3840764192
iter      3: update = [ 2.16400435e-02 -6.418715

iter      1: update = [-0.21012904 -0.28342977 -0.10044462  0.5261845  -0.16268653 -0.05654894
  0.17442952]
iter      1: loss = 6.3778558470
iter      2: update = [0.00639962 0.01758489 0.00445487 0.17831296 0.01050296 0.00343006
 0.01645775]
iter      2: loss = 6.3768445703
iter      3: update = [-4.03335954e-05 -1.60930585e-04 -1.01983656e-04  1.42875982e-02
 -1.35681676e-04 -4.66000056e-05 -6.48337120e-05]
iter      3: loss = 6.3768396074
iter      4: update = [ 6.35865285e-08  1.52706427e-07 -6.82757174e-09  8.41478400e-05
 -4.70198627e-07  2.13762107e-07 -9.65889959e-07]
iter      4: loss = 6.3768396073
iter      4: optimization converged
iter      1: update = [-0.48633995  0.28105732  0.17808788 -0.2257041   0.27302858  0.75303937
 -0.37620369]
iter      1: loss = 6.3722481001
iter      2: update = [ 0.03409685  0.10345175 -0.00379671  0.01504192 -0.00807923  0.00867858
  0.02539601]
iter      2: loss = 6.3719214557
iter      3: update = [-2.33567723e-04  3.14319940e-03  1.32140

iter      1: update = [-0.23863867 -0.2698439  -0.46113681  0.24536945 -0.25747685  0.77888184
 -0.13534924  0.04734721 -0.11904467 -0.19882451 -0.42145195]
iter      1: loss = 6.3580610258
iter      2: update = [ 0.00972991  0.00897263  0.06177772  0.03938056  0.00520168  0.2189634
  0.01028499 -0.00550297  0.00406086  0.07523007  0.01909453]
iter      2: loss = 6.3561970463
iter      3: update = [ 3.68520135e-05 -1.05517248e-04  7.90306808e-04 -2.34509380e-04
 -1.79177755e-04  1.83908909e-02  2.38607934e-04 -1.57502415e-04
  9.55459816e-05  2.48818427e-03 -6.77243096e-04]
iter      3: loss = 6.3561884527
iter      4: update = [-4.10797122e-08  2.78078635e-07  4.63102678e-07  2.00489017e-06
 -6.86363354e-09  1.38234778e-04 -7.64210023e-07 -6.37543722e-07
 -2.00817594e-07  6.05517060e-07 -4.38625276e-07]
iter      4: loss = 6.3561884522
iter      4: optimization converged
iter      1: update = [-0.03452508 -0.21088972 -0.00305844 -0.31423919 -0.05916029  0.43428842
 -0.22458671 -0.2322

iter      1: update = [-0.23149615 -0.29353974 -0.46867686  0.2292815   0.00892253 -0.26544392
  0.94907129  0.23184157 -0.01076436 -0.23973186  0.05753068 -0.10375915
 -0.22360552 -0.36610098]
iter      1: loss = 6.3530239362
iter      2: update = [ 0.008738   -0.0052758   0.03374512  0.02198659  0.01547657 -0.01553034
  0.18823272  0.00291869  0.00959013  0.02615358  0.01084121  0.0004906
  0.0571324   0.01212513]
iter      2: loss = 6.3513261988
iter      3: update = [ 4.43161847e-05 -8.55774714e-05  4.99329561e-04 -1.08586729e-04
  5.17183306e-05  6.48946088e-05  1.74092272e-02 -1.70935020e-05
 -5.40985114e-04  2.45092619e-05 -2.23285860e-04  2.42325102e-04
  2.09929522e-03 -4.23259636e-04]
iter      3: loss = 6.3513185508
iter      4: update = [-1.42957416e-09  2.71150199e-07  3.33305535e-07  1.54255514e-06
  7.31959565e-08 -1.39851171e-07  1.24703222e-04  5.83997051e-08
  1.43746193e-07 -7.11912415e-07 -6.83426443e-07 -2.98200476e-07
 -4.23741859e-08 -4.72559101e-07]
iter      4:

...

iter      5: update = [-8.40674411e-08 -3.10751781e-07  3.58721515e-07 -1.50233910e-07
 -7.96356067e-08  3.91195694e-07 -8.82698601e-08  1.02973427e-09
  1.47347339e-07  1.77469239e-07  2.65603144e-07  1.79201125e-07
  7.40523873e-04  1.35735487e-08  7.92735899e-08 -2.97391174e-08
 -2.74855972e-08 -7.65373524e-08  8.68208452e-08 -6.81743434e-08
  7.40523878e-04  1.38057609e-08  7.22948921e-08 -9.01036366e-08
 -1.10677769e-08 -9.42190240e-08  9.36000424e-08  3.74680298e-08
  1.30265181e-07  1.06203036e-07 -1.69945867e-07 -1.11471200e-07
 -1.11469983e-07  2.98004073e-07  0.00000000e+00]
iter      5: loss = 6.3454524346
iter      6: update = [-5.15219757e-11 -1.98727873e-10  2.32446177e-10 -9.56807445e-11
 -5.20735365e-11  2.50243320e-10 -5.41155318e-11  5.15485817e-13
  9.72550638e-11  1.18660923e-10  1.78548058e-10  1.20187253e-10
  5.20424147e-07  9.28181796e-12  5.36052384e-11 -1.47669987e-11
 -1.87343338e-11 -4.83934264e-11  5.82807321e-11 -4.51056835e-11
  5.20424291e-07  6.36636456

iter      5: update = [-8.28792103e-08 -3.09480225e-07  3.58630493e-07 -1.49556520e-07
 -8.09808485e-08  3.89768573e-07 -8.29812497e-08  6.75404713e-11
  1.55303512e-07  1.79321783e-07  2.73515451e-07  1.82542403e-07
  7.39070367e-04  7.49610326e-09  9.11327844e-08 -2.30403263e-08
 -2.90453490e-08 -5.72067003e-08  9.33777841e-08 -7.64525541e-08
  7.39070191e-04  8.62089976e-09  3.84512902e-08 -6.84809336e-08
 -1.83453995e-08 -9.63847172e-08  9.26794457e-08  3.08252871e-08
  1.21370500e-07  8.44819290e-08 -1.78801278e-07 -1.12060830e-07
 -9.75337111e-08 -1.12059613e-07  2.99768812e-07  0.00000000e+00]
iter      5: loss = 6.3449110076
iter      6: update = [-5.07051475e-11 -1.97602363e-10  2.31936872e-10 -9.51090808e-11
 -5.28246594e-11  2.48834773e-10 -5.10203158e-11 -2.08731298e-13
  1.02142067e-10  1.19256767e-10  1.82586664e-10  1.21152478e-10
  5.18398970e-07  5.38122372e-12  6.05316592e-11 -1.14342234e-11
 -2.00230682e-11 -3.62016827e-11  6.19909413e-11 -5.03851131e-11
  5.18405039

iter      5: update = [-8.91022492e-08 -3.17074892e-07  3.48116300e-07 -1.50622128e-07
 -7.99214665e-08  3.88122969e-07 -5.90396551e-08 -8.81908668e-08
  5.43128073e-09  1.62446303e-07  1.83171666e-07  2.85590642e-07
  1.83905318e-07  7.38138822e-04  1.03533349e-08  9.55535753e-08
 -1.74441945e-08 -3.49793890e-08 -6.10099432e-08  8.22123313e-08
 -7.71402152e-08  7.38138987e-04  1.36547351e-08  4.32697662e-08
 -6.88555345e-08 -1.41018169e-08 -9.65026818e-08  9.90436911e-08
  3.49293887e-08  1.27496024e-07  8.85552943e-08 -1.96313398e-07
 -1.05088670e-07 -1.07482143e-07 -1.05088670e-07  3.24313356e-07
  0.00000000e+00]
iter      5: loss = 6.3448785828
iter      6: update = [-5.37906379e-11 -2.01519259e-10  2.25132671e-10 -9.51130436e-11
 -5.23064821e-11  2.46717193e-10 -3.45216873e-11 -5.41913200e-11
  2.88405428e-12  1.05993062e-10  1.21142279e-10  1.88936086e-10
  1.21484029e-10  5.17086509e-07  6.94251419e-12  6.29296041e-11
 -8.16904916e-12 -2.34190854e-11 -3.83153806e-11  5.52917605

iter      4: update = [-4.23223832e-06 -1.68933045e-05  2.20509099e-05 -9.73277692e-06
 -4.44445903e-06  2.41043597e-05 -3.92273705e-06  3.68315636e-07
  1.92466166e-06  1.47467561e-05  1.64441103e-05  2.24271559e-05
  1.24106146e-05  2.73671319e-02  1.78192213e-06  5.61272649e-06
  2.81130855e-06 -5.92410550e-06 -5.79259272e-06  5.45609806e-06
 -5.64594540e-06  2.73671304e-02 -4.43111070e-07 -1.48437901e-06
 -3.93683819e-06 -3.97715477e-06 -8.37620770e-06  4.52049508e-06
  9.87817232e-07 -9.21411115e-06  9.22120993e-06  4.32511629e-06
 -1.97600514e-05  3.87252669e-05 -2.75418315e-06  3.87252779e-05
  1.96474024e-05  0.00000000e+00]
iter      4: loss = 6.3445209171
iter      5: update = [-8.76052046e-08 -3.15332161e-07  3.45106514e-07 -1.49275679e-07
 -7.95528700e-08  3.84734427e-07 -5.73878165e-08 -8.91367653e-08
  5.35350893e-09  1.73028794e-07  1.89589300e-07  2.86369876e-07
  1.90276235e-07  7.33594206e-04  8.58010230e-09  1.04248036e-07
 -8.06697147e-09 -3.35913953e-08 -5.74571225

iter      1: update = [-0.08334616 -0.01233131  0.04822813  0.0817537  -0.16592342 -0.24865089
 -0.02060693  0.36714194  0.09540419 -0.09253471 -0.56809438 -0.4172228
  0.01741844 -1.14972595  0.20021176 -0.39531152  0.26810362  0.13119294
  0.01338649  0.2658855   0.46453718 -1.14972595 -0.09151735  0.58937144
  0.15755395 -0.1838418   0.11000183 -0.12879949  0.13989143  0.09421702
  0.21735964 -0.1844222   0.08750956  0.2417159   0.2762501   0.25243631
  0.2762501  -1.02598974  0.        ]
iter      1: loss = 6.3531664221
iter      2: update = [ 9.83805033e-03  9.46911308e-03  1.18428080e-02  4.27476277e-03
  8.31569991e-03  1.76258949e-03 -8.06370663e-03  8.91712006e-02
  1.37992222e-02  4.69379576e-02  5.17737481e-02  6.96905988e-02
  1.20962648e-01  3.87465204e-01  1.72405299e-02  5.29247575e-02
  4.65787322e-02 -8.45671951e-03  3.09710443e-02  1.45406970e-01
 -1.08488595e-02  3.87465207e-01 -3.24612313e-02 -7.62470473e-02
 -2.55911639e-02 -2.37013334e-03 -3.34825535e-02  3.493108

In [64]:
gcv.best_params_

{'select__k': 1}

In [46]:
pipe.set_params(**gcv.best_params_)
pipe.fit(data_x, data_y)

iter      1: update = [-0.01574184  0.53942117  0.00081675  0.14268407  0.01968065 -0.282868
 -0.03143889 -0.19215379  0.56981461 -0.09358615]
iter      1: loss = 6.7606396809
iter      2: update = [ 2.08459156e-04 -1.68132389e-03  2.77867594e-05  5.58092324e-03
  3.96115855e-03  1.61643247e-02 -1.71670454e-02 -2.67975659e-02
  1.87454127e-01  5.78788522e-03]
iter      2: loss = 6.7597342095
iter      3: update = [-3.32845509e-06  6.10995158e-05  2.92797413e-08  4.64111823e-05
 -1.17654290e-04 -1.59813173e-04  7.30261146e-05  1.71564881e-04
  1.40056648e-02  2.43563468e-06]
iter      3: loss = 6.7597297647
iter      4: update = [-1.80381049e-08 -2.73325338e-07 -7.93919070e-11  1.94033510e-07
 -1.14303595e-07 -8.07161252e-08 -3.61109686e-07 -5.88288841e-07
  8.43518636e-05  1.55527620e-07]
iter      4: loss = 6.7597297646
iter      4: optimization converged


Pipeline(memory=None,
     steps=[('select', SelectKBest(k=10,
      score_func=<function fit_and_score_features at 0x7f82f85371e0>)), ('model', CoxPHSurvivalAnalysis(alpha=1e-06, n_iter=100, tol=1e-09, verbose=10))])

In [47]:
pipe.score(data_x, data_y)

0.5871271758141084

In [48]:
pd.DataFrame(gcv.cv_results_).sort_values(by='mean_test_score', ascending=False)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_select__k,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
9,8.932500,0.095792,0.036739,0.000311,10,{'select__k': 10},0.547882,0.568809,0.533020,0.549904,0.014680,1,0.599506,0.578039,0.610643,0.596063,0.013531
8,8.673455,0.197501,0.034316,0.000426,9,{'select__k': 9},0.549493,0.560509,0.532990,0.547664,0.011309,2,0.598978,0.577984,0.610673,0.595878,0.013524
11,8.810566,0.146123,0.039854,0.001538,12,{'select__k': 12},0.543657,0.569798,0.525528,0.546328,0.018172,3,0.607869,0.582864,0.614041,0.601592,0.013480
10,8.899614,0.204316,0.038404,0.001487,11,{'select__k': 11},0.547839,0.569282,0.518866,0.545329,0.020659,4,0.599588,0.581886,0.611521,0.597665,0.012175
12,8.803736,0.199515,0.040824,0.001167,13,{'select__k': 13},0.544272,0.557451,0.529813,0.543845,0.011287,5,0.613126,0.582587,0.614929,0.603547,0.014839
13,8.877484,0.152291,0.051685,0.013087,14,{'select__k': 14},0.543903,0.558141,0.529073,0.543706,0.011868,6,0.615773,0.583032,0.615115,0.604640,0.015282
14,8.930264,0.117295,0.045663,0.002817,15,{'select__k': 15},0.543500,0.556759,0.529149,0.543136,0.011274,7,0.616395,0.583520,0.615147,0.605021,0.015212
7,8.798854,0.069682,0.033998,0.000848,8,{'select__k': 8},0.549193,0.548240,0.526258,0.541230,0.010594,8,0.597671,0.572652,0.606492,0.592272,0.014333
2,8.481652,0.153073,0.015252,0.000460,3,{'select__k': 3},0.544902,0.529252,0.549035,0.541063,0.008520,9,0.582867,0.561930,0.584673,0.576490,0.010322
1,8.484536,0.222326,0.014485,0.000148,2,{'select__k': 2},0.545105,0.543323,0.532538,0.540322,0.005552,10,0.564752,0.557328,0.569653,0.563911,0.005067


In [ ]:
scores = fit_and_score_features(data_x.values, data_y)
pd.Series(scores, index=data_x.columns)